In [3]:
%%bash
pip install pdfminer
pip install pdfminer.six

  Using cached pdfminer-20191125-py3-none-any.whl
  Using cached pycryptodome-3.15.0-cp35-abi3-manylinux2010_x86_64.whl (2.3 MB)
  Using cached pdfminer.six-20220524-py3-none-any.whl (5.6 MB)


In [30]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import os
import json
from io import StringIO
from pdfminer.high_level import extract_text_to_fp
from pdfminer.high_level import extract_pages
from pdfminer.layout import LAParams
from pdfminer.layout import LTTextContainer
from json import JSONEncoder
from collections import namedtuple

import pandas as pd

## Extracting Salgsoppgave PDFs from DNB Eiendom

In [27]:
## done with leiligheter
## html_doc = open("./DNB/Boliger til salgs _ DNB Eiendom.html")

## eneboliger
html_doc = open("./DNB/Boliger til salgs _ DNB Eiendom_Enebolig.html")

## rekkehus
#html_doc = open("./DNB/Boliger til salgs _ DNB EiendomRekkehus.html")

## tomansbolig
##html_doc = open("./DNB/Boliger til salgs _ DNB EiendomTomansbolig.html")
soup = BeautifulSoup(html_doc, 'html.parser')

In [28]:
## extract bolig links
bolig_links = []
for link in soup.find_all('a'):
    url = link.get('href')
    if "/bolig/" in url:
        bolig_links.append(url)

print(len(bolig_links))

571


In [29]:
## download html and extract pdf links
pdf_links = []
for bolig_link in bolig_links:
    r = requests.get(bolig_link)
    soup = BeautifulSoup(r.text, 'html.parser')
    for link in soup.find_all('a'):
        url = link.get('href')
        if ".pdf" in url:
            pdf_links.append(url)
            
##should be as many as from the line before
print(len(pdf_links))
print(pdf_links[0])
print(pdf_links[100])

KeyboardInterrupt: 

In [ ]:
## Save PDFs in the folder function
#leiligheter
#folderUrl = "./DNB/PDFSLeilighet/"

#eneboliger
folderUrl = "./DNB/PDFSEnebolig/"

#rekkehus
#folderUrl = "./DNB/PDFSRekkehus/"

#Tomansbolig
#folderUrl = "./DNB/PDFSTomansbolig/"

def download_file(download_url, filename):
    if os.path.exists(folderUrl + filename + ".pdf"):
        print("exists")
        return
    response = urllib.request.urlopen(download_url)    
    file = open(folderUrl + filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()
    print(f"made file {filename}")

In [ ]:
for pdf_link in pdf_links:
    try:
        name = pdf_link.split("/")[-1].split(".")[0]
        download_file(pdf_link,name)
    except:
        print(f"file {name} failed")

In [ ]:
number_of_files = len(os.listdir(folderUrl))
print(number_of_files)

In [ ]:
class Bolig:
        prisantydning = []
        byggeaar = []
        bruksareal = []
        number_of_rooms = []
        heating = []
        location = []
        energy_label = []
        property_type_description = []

In [ ]:
##  Extracting info without Azure?
#leiligheter
#folderUrl = "./DNB/PDFSLeilighet/"
#folderOutputUrl = "./DNB/JSON/Untreated/Leilighet/"

#eneboliger
folderUrl = "./DNB/PDFSEnebolig/"
folderOutputUrl = "./DNB/JSON/Untreated/Enebolig/"

#rekkehus
#folderUrl = "./DNB/PDFSTomansbolig/"
#folderOutputUrl = "./DNB/JSON/Untreated/Tomansbolig/"

i = 0
for file in os.listdir(folderUrl):
    i+=1
    print(f"file number {i} {file}")
    if os.path.exists(folderOutputUrl + file.split(".")[0] +".json"):
            print("exists")
            continue
    with open(folderUrl + file, 'rb') as f:
        bolig = Bolig()
        bolig.prisantydning = []
        bolig.byggeaar = []
        bolig.bruksareal = []
        bolig.number_of_rooms = []
        bolig.heating = []
        bolig.location = []
        bolig.energy_label = []
        bolig.property_type_description = []
        bolig.property_type_description.append('enebolig')
        try:
            for page_layout in extract_pages(f):
                for element in page_layout:
                    if isinstance(element, LTTextContainer):
                        # vi trenger prisantydning
                        if 'prisantydning' in str.lower(element.get_text()):
                            bolig.prisantydning.append(str.lower(element.get_text()).split('\n'))
                        if 'byggeår' in str.lower(element.get_text()):
                            bolig.byggeaar.append(str.lower(element.get_text()).split('\n'))
                        if 'bruksareal' in str.lower(element.get_text()):
                            bolig.bruksareal.append(str.lower(element.get_text()).split('\n'))
                        if 'antall rom' in str.lower(element.get_text()):
                            bolig.number_of_rooms.append(str.lower(element.get_text()).split('\n'))
                        if 'oppvarming' in str.lower(element.get_text()):
                            bolig.heating.append(str.lower(element.get_text()).split('\n'))
                        if 'adresse' or 'kommune' in str.lower(element.get_text()):
                            bolig.location.append(str.lower(element.get_text()).split('\n'))
                        if 'energikarakter' or 'energifarge' in str.lower(element.get_text()):
                            bolig.energy_label.append(str.lower(element.get_text()).split('\n'))
        except:
            print(f"could not extract from document number {i}")
        
        jsonStr = json.dumps(bolig.__dict__,indent=4,sort_keys=True,ensure_ascii=False)
        with open(folderOutputUrl + file.split(".")[0] +".json", 'w', encoding="utf-8") as outfile:
                outfile.write(jsonStr)
                outfile.close()
    

In [ ]:
number_of_files = len(os.listdir(folderOutputUrl))
print(number_of_files)

# Dynamically download the PDF from the public DNB API

In [31]:
#loop through the api 
#settings
url_base='https://dnbeiendom.no/api/v1/search/properties?LocationId=2fd6d7f6-4bad-4c3c-b940-08d59bb7ce4c&OrderByd=Published%20desc&pageSize=100&pageNumber='
response=requests.get(url_base+'i').json()
totalPages = response['pagination']['totalPages']
features=response['result'][0].keys()


raw_data=[]
for i in range(1,totalPages+1):
    response=requests.get(url_base+'i').json()

    for element in response['result']:
        raw_data.append(element)

    print(f"page {i}")



page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17


In [32]:
features

dict_keys(['id', 'assignmentNumber', 'mainImage', 'propertyType', 'propertyBaseType', 'price', 'collectiveDept', 'collectiveDeptFrom', 'collectiveDeptTo', 'totalPrice', 'totalPriceFrom', 'totalPriceTo', 'priceFrom', 'priceTo', 'purchaseCostsAmount', 'purchaseCostsAmountFrom', 'purchaseCostsAmountTo', 'originalExpensesPrice', 'rentPrMonth', 'primaryRoomArea', 'primaryRoomAreaFrom', 'primaryRoomAreaTo', 'usableArea', 'numberOfBedRooms', 'published', 'showings', 'ownership', 'siteOwnership', 'longitude', 'latitude', 'country', 'countryId', 'county', 'countyId', 'municipality', 'municipalityId', 'city', 'cityId', 'zipCode', 'zipCodeId', 'district', 'districtId', 'street', 'streetId', 'changed', 'created', 'images'])

In [33]:
#filtering of the data 
data=[]

for el in raw_data:

    id_ad = el['id']
    propertyType = el['propertyType']
    price = el['price']
    numberOfBedRooms = el['numberOfBedRooms']
    usableArea = el['usableArea']
    ownershipType = el['ownership']
    longitude = el['longitude']
    latitude = el['latitude']
    city = str.lower(el['city'])
    zipCode = el['zipCode']
    street = str.lower(el['street'])

    data.append([id_ad,propertyType,price,numberOfBedRooms,usableArea,ownershipType,longitude,latitude,street,city,zipCode])

    


In [35]:
len(raw_data)

1700

In [8]:
len(data)

1700

In [40]:
df = pd.DataFrame(data)
df.columns=['id_ad','propertyType','price','numberOfBedRooms','usableArea','ownershipType','longitude','latitude','street','city','zipCode']

In [41]:
df.head(3)

,id_ad,propertyType,price,numberOfBedRooms,usableArea,ownershipType,longitude,latitude,street,city,zipCode
0,2562bd65-ffb6-400a-5761-08da664090a3,1,3490000,4,174.0,0,15.389475,68.679817,karl reinsnes vei 9,sortland,8403
1,07f73128-bee8-46b5-1bf4-08da664756c1,1,5700000,5,232.0,0,7.187793,62.752882,tyttebærsvingen 43,molde,6429
2,b534b37d-113d-4265-5760-08da664090a3,24,1230000,1,48.0,1,9.656663,59.128182,sykehusvegen 25c,porsgrunn,3924


### upload the data 

In [42]:
db = client.DnbData
collection=db.basicinfo

### upload the data to MongoDB

In [44]:
records = json.loads(df.T.to_json()).values()

In [46]:
db.myCollection.insert_many(records)